In [1]:
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
import math

In [2]:
token = "AAAAAAAAAAAAAAAAAAAAAJB8MgEAAAAAxYdk05eK8SzCl%2FPfllDwicULg%2FE%3DhwOjnjhNk4naJ3F0PTf71ybLUXgtxl3zEwUBNN32HP6q7ZNKbC"

In [3]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [4]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    #'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'created_at',
                    #'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    #'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}
                   }
    return (search_url, query_params)

In [5]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [6]:
def create_end_time(year, month, firstday, lastday, high, current):
    currentday = lastday
    if current == 0:
        currentday = firstday + 1
    elif high > current:
        totaldays = lastday - firstday
        deltadays = math.ceil(totaldays / high)
        currentday = firstday + deltadays * current
        if currentday > lastday:
            currentday = lastday
    if month < 10:
        month = "0{}".format(month)
    if currentday < 10:
        currentday = "0{}".format(currentday)
    endtime = "{}-{}-{}T00:00:00.000Z".format(year, month, currentday)
    return endtime

In [7]:
def create_start_time(year, month, day):
    if month < 10:
        month = "0{}".format(month)
    if day < 10:
        day = "0{}".format(day)
    return "{}-{}-{}T00:00:00.000Z".format(year, month, day)

In [8]:
def create_df(year, month, firstday, lastday, searchsize, csv_name):
    bearer_token = token
    headers = create_headers(bearer_token)
    keyword = '(covid OR coronavirus) vaccine -is:retweet'
    max_results = 500
    nexttoken = None
    df = None
    df = pd.DataFrame(data={'id': [], 'text': []})
    start_time = create_start_time(year, month, firstday)
    for i in range(searchsize):
        #if i % 4 == 0 and i > 1:
            #time.sleep(910)
        print(create_end_time(year, month, firstday, lastday, searchsize, i))
        url = create_url(keyword, start_time,create_end_time(year, month, firstday, lastday, searchsize, i), max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1])
        #nexttoken = json_response['meta']['next_token']
        #print(nexttoken)
        tempdf = pd.DataFrame.from_dict(json_response['data'])
        df = df.append(tempdf, ignore_index=True)
        time.sleep(1)
      
    print(df)
    df = df.drop_duplicates(subset=['text'])
    print(df)
    df.to_csv(csv_name)
    return df

In [9]:
print(create_end_time(2020, 3, 1, 31, 10, 9))

2020-03-28T00:00:00.000Z


In [10]:
create_df(2020, 3, 15, 31, 10, 'march20.csv')

2020-03-16T00:00:00.000Z
Endpoint Response Code: 200
2020-03-17T00:00:00.000Z
Endpoint Response Code: 200
2020-03-19T00:00:00.000Z
Endpoint Response Code: 200
2020-03-21T00:00:00.000Z
Endpoint Response Code: 200
2020-03-23T00:00:00.000Z
Endpoint Response Code: 200
2020-03-25T00:00:00.000Z
Endpoint Response Code: 200
2020-03-27T00:00:00.000Z
Endpoint Response Code: 200
2020-03-29T00:00:00.000Z
Endpoint Response Code: 200
2020-03-31T00:00:00.000Z
Endpoint Response Code: 200
2020-03-31T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1239340599356071938  @MysterySolvent 🤮🤮🤮🤮🤮🤮@IvankaTrump It’s a give...   
1     1239340586454462464  Donald Trump tried to convince German scientis...   
2     1239340580200685568  JUST SAW SOMEONE SAY THAT WE ARE THE VIRUS AND...   
3     1239340579714195456  Government official: Coronavirus vaccine trial...   
4     1239340570574835712  🚨#DEVELOPING A clinical trial evaluating a 

,id,text,created_at,withheld
0,1239340599356071938,@MysterySolvent 🤮🤮🤮🤮🤮🤮@IvankaTrump It’s a give...,2020-03-15T23:59:59.000Z,NaN
1,1239340586454462464,Donald Trump tried to convince German scientis...,2020-03-15T23:59:56.000Z,NaN
2,1239340580200685568,JUST SAW SOMEONE SAY THAT WE ARE THE VIRUS AND...,2020-03-15T23:59:54.000Z,NaN
3,1239340579714195456,Government official: Coronavirus vaccine trial...,2020-03-15T23:59:54.000Z,NaN
4,1239340570574835712,🚨#DEVELOPING A clinical trial evaluating a vac...,2020-03-15T23:59:52.000Z,NaN
...,...,...,...,...
4330,1244753521934766083,Fingers crossed! And thanks to Dan Barouch and...,2020-03-30T22:29:00.000Z,NaN
4331,1244753513009369089,@sam896luc @Swapnil74219485 @cjwerleman @dwnew...,2020-03-30T22:28:58.000Z,NaN
4332,1244753439193735169,"This Thursday, 4/2 at 10 a.m. ET, I'll be co-h...",2020-03-30T22:28:40.000Z,NaN
4333,1244753417697923075,@JoshMayhugh1 @FOX13News That’s incredible. Th...,2020-03-30T22:28:35.000Z,NaN


In [11]:
create_df(2020, 4, 1, 30, 10, 'april20.csv')

2020-04-02T00:00:00.000Z
Endpoint Response Code: 200
2020-04-04T00:00:00.000Z
Endpoint Response Code: 200
2020-04-07T00:00:00.000Z
Endpoint Response Code: 200
2020-04-10T00:00:00.000Z
Endpoint Response Code: 200
2020-04-13T00:00:00.000Z
Endpoint Response Code: 200
2020-04-16T00:00:00.000Z
Endpoint Response Code: 200
2020-04-19T00:00:00.000Z
Endpoint Response Code: 200
2020-04-22T00:00:00.000Z
Endpoint Response Code: 200
2020-04-25T00:00:00.000Z
Endpoint Response Code: 200
2020-04-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1245501140529221633  #Coronavirus – No Vaccine Is Needed to Cure It...   
1     1245501080907046912  CSIRO tests two potential COVID-19 vaccines in...   
2     1245501069012037632  We have to apologise... in between funding the...   
3     1245501043145834496  @kk_OEG @adamboultonSKY I sincerely doubt the ...   
4     1245501029543788544  @ComposedVirgil @tinpansaf @FaIse9Firmino T

,id,text,created_at,withheld
0,1245501140529221633,#Coronavirus – No Vaccine Is Needed to Cure It...,2020-04-01T23:59:46.000Z,NaN
1,1245501080907046912,CSIRO tests two potential COVID-19 vaccines in...,2020-04-01T23:59:32.000Z,NaN
2,1245501069012037632,We have to apologise... in between funding the...,2020-04-01T23:59:29.000Z,NaN
3,1245501043145834496,@kk_OEG @adamboultonSKY I sincerely doubt the ...,2020-04-01T23:59:23.000Z,NaN
4,1245501029543788544,@ComposedVirgil @tinpansaf @FaIse9Firmino Ther...,2020-04-01T23:59:20.000Z,NaN
...,...,...,...,...
4733,1254910111887380489,From the @nytimes \n\nhttps://t.co/eSlu8jVW0g ...,2020-04-27T23:07:40.000Z,NaN
4734,1254910093419839494,Column: Here's why a COVID-19 vaccine could en...,2020-04-27T23:07:36.000Z,NaN
4735,1254910028101742592,Chinese biotech firm says coronavirus vaccine ...,2020-04-27T23:07:20.000Z,NaN
4736,1254909959868907521,NEW: The race for coronavirus vaccine faces ea...,2020-04-27T23:07:04.000Z,NaN


In [12]:
create_df(2020, 5, 1, 31, 10, 'may20.csv')

2020-05-02T00:00:00.000Z
Endpoint Response Code: 200
2020-05-04T00:00:00.000Z
Endpoint Response Code: 200
2020-05-07T00:00:00.000Z
Endpoint Response Code: 200
2020-05-10T00:00:00.000Z
Endpoint Response Code: 200
2020-05-13T00:00:00.000Z
Endpoint Response Code: 200
2020-05-16T00:00:00.000Z
Endpoint Response Code: 200
2020-05-19T00:00:00.000Z
Endpoint Response Code: 200
2020-05-22T00:00:00.000Z
Endpoint Response Code: 200
2020-05-25T00:00:00.000Z
Endpoint Response Code: 200
2020-05-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1256372692074336263  @EconguyRosie It recouped the ‘99 ‘00 avg arou...   
1     1256372659564294144  @SteveBrownBC There's a vaccine for flus and n...   
2     1256372637632249862  The little detail that proponents of ‘natural’...   
3     1256372609815588866  Landmark partnership announced for development...   
4     1256372504001753090  Tears of joy role down from the people of T

,id,text,created_at
0,1256372692074336263,@EconguyRosie It recouped the ‘99 ‘00 avg arou...,2020-05-01T23:59:26.000Z
1,1256372659564294144,@SteveBrownBC There's a vaccine for flus and n...,2020-05-01T23:59:18.000Z
2,1256372637632249862,The little detail that proponents of ‘natural’...,2020-05-01T23:59:13.000Z
3,1256372609815588866,Landmark partnership announced for development...,2020-05-01T23:59:07.000Z
4,1256372504001753090,Tears of joy role down from the people of Taxa...,2020-05-01T23:58:41.000Z
...,...,...,...
4814,1265773326544273410,What we must keep in mind is that the market r...,2020-05-27T22:34:12.000Z
4815,1265773317044277253,Phony ‘Israeli’ coronavirus vaccine peddled in...,2020-05-27T22:34:10.000Z
4816,1265773282940391430,Most Americans expect a COVID-19 vaccine withi...,2020-05-27T22:34:02.000Z
4817,1265773235146231808,@BobProbert241 @UE_Loyalist @birdintexas @ever...,2020-05-27T22:33:50.000Z


In [13]:
create_df(2020, 6, 1, 30, 10, 'june20.csv')

2020-06-02T00:00:00.000Z
Endpoint Response Code: 200
2020-06-04T00:00:00.000Z
Endpoint Response Code: 200
2020-06-07T00:00:00.000Z
Endpoint Response Code: 200
2020-06-10T00:00:00.000Z
Endpoint Response Code: 200
2020-06-13T00:00:00.000Z
Endpoint Response Code: 200
2020-06-16T00:00:00.000Z
Endpoint Response Code: 200
2020-06-19T00:00:00.000Z
Endpoint Response Code: 200
2020-06-22T00:00:00.000Z
Endpoint Response Code: 200
2020-06-25T00:00:00.000Z
Endpoint Response Code: 200
2020-06-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1267606590682587136  @Drmuzoic @NCDCgov Its been said that Covid wi...   
1     1267606575004278784  Great outcome for our @UQ_News COVID-19 #vacci...   
2     1267606238495309824        @anpanboos And invent the vaccine for covid   
3     1267606197168996352  @Rene_Bogoshi @DineoDeeMash2 Now we know long ...   
4     1267606053560221699  In this interview he is very specific about

,id,text,created_at
0,1267606590682587136,@Drmuzoic @NCDCgov Its been said that Covid wi...,2020-06-01T23:58:56.000Z
1,1267606575004278784,Great outcome for our @UQ_News COVID-19 #vacci...,2020-06-01T23:58:53.000Z
2,1267606238495309824,@anpanboos And invent the vaccine for covid,2020-06-01T23:57:32.000Z
3,1267606197168996352,@Rene_Bogoshi @DineoDeeMash2 Now we know long ...,2020-06-01T23:57:23.000Z
4,1267606053560221699,In this interview he is very specific about th...,2020-06-01T23:56:48.000Z
...,...,...,...
4815,1276989714524966913,Evidence-free decision making is the Rule in b...,2020-06-27T21:24:07.000Z
4816,1276989711320571910,@realDonaldTrump And you said there was only 1...,2020-06-27T21:24:07.000Z
4817,1276989488963686401,@LubaK19260165 @Barnes_Law Has Texas used the ...,2020-06-27T21:23:14.000Z
4818,1276989397544529920,"""#Coronavirus Responses Highlight How Humans H...",2020-06-27T21:22:52.000Z


In [14]:
create_df(2020, 7, 1, 31, 10, 'july20.csv')

2020-07-02T00:00:00.000Z
Endpoint Response Code: 200
2020-07-04T00:00:00.000Z
Endpoint Response Code: 200
2020-07-07T00:00:00.000Z
Endpoint Response Code: 200
2020-07-10T00:00:00.000Z
Endpoint Response Code: 200
2020-07-13T00:00:00.000Z
Endpoint Response Code: 200
2020-07-16T00:00:00.000Z
Endpoint Response Code: 200
2020-07-19T00:00:00.000Z
Endpoint Response Code: 200
2020-07-22T00:00:00.000Z
Endpoint Response Code: 200
2020-07-25T00:00:00.000Z
Endpoint Response Code: 200
2020-07-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1278478468653568000            I hope we get a vaccine  soon for covid   
1     1278478407035162624  ICYMI: Japanese University says its isolated a...   
2     1278478364509175815  Actually good news since COVID-19 WILL continu...   
3     1278478299505864704  so you're telling me they can recycle toilet w...   
4     1278478291813482498  UN council since  everything is done with t

,id,text,created_at
0,1278478468653568000,I hope we get a vaccine soon for covid,2020-07-01T23:59:54.000Z
1,1278478407035162624,ICYMI: Japanese University says its isolated a...,2020-07-01T23:59:39.000Z
2,1278478364509175815,Actually good news since COVID-19 WILL continu...,2020-07-01T23:59:29.000Z
3,1278478299505864704,so you're telling me they can recycle toilet w...,2020-07-01T23:59:14.000Z
4,1278478291813482498,UN council since everything is done with the ...,2020-07-01T23:59:12.000Z
...,...,...,...
4807,1287891025692958721,Are you considering volunteering to be a parti...,2020-07-27T23:22:03.000Z
4808,1287890996186013698,@sburke85 @TeamTrump @realDonaldTrump I am sur...,2020-07-27T23:21:56.000Z
4809,1287890965001371650,More info https://t.co/45Z0WmqKMg,2020-07-27T23:21:48.000Z
4810,1287890935574081539,@Mitch_Rupert Amen bro...nothing happens until...,2020-07-27T23:21:41.000Z


In [15]:
create_df(2020, 8, 1, 31, 10, 'aug20.csv')

2020-08-02T00:00:00.000Z
Endpoint Response Code: 200
2020-08-04T00:00:00.000Z
Endpoint Response Code: 200
2020-08-07T00:00:00.000Z
Endpoint Response Code: 200
2020-08-10T00:00:00.000Z
Endpoint Response Code: 200
2020-08-13T00:00:00.000Z
Endpoint Response Code: 200
2020-08-16T00:00:00.000Z
Endpoint Response Code: 200
2020-08-19T00:00:00.000Z
Endpoint Response Code: 200
2020-08-22T00:00:00.000Z
Endpoint Response Code: 200
2020-08-25T00:00:00.000Z
Endpoint Response Code: 200
2020-08-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1289712469045211136  @Watchman1209007 @MelissaAFrancis And when do ...   
1     1289712353542455299  @Matt_Hopcraft @CapybaraHappy @InsidersABC @Ca...   
2     1289712276623089665  La vacuna de coronavirus Moderna causa efectos...   
3     1289712231530274816  Great thread on the challenges of vaccine deve...   
4     1289712212232175617  @cnnphilippines THE ANATHEMA OF TRUTH: CHIN

,id,text,created_at
0,1289712469045211136,@Watchman1209007 @MelissaAFrancis And when do ...,2020-08-01T23:59:48.000Z
1,1289712353542455299,@Matt_Hopcraft @CapybaraHappy @InsidersABC @Ca...,2020-08-01T23:59:21.000Z
2,1289712276623089665,La vacuna de coronavirus Moderna causa efectos...,2020-08-01T23:59:03.000Z
3,1289712231530274816,Great thread on the challenges of vaccine deve...,2020-08-01T23:58:52.000Z
4,1289712212232175617,@cnnphilippines THE ANATHEMA OF TRUTH: CHINESE...,2020-08-01T23:58:47.000Z
...,...,...,...
4759,1299108795730202625,@Atul_Gawande @CDCgov We can’t trust integrity...,2020-08-27T22:17:27.000Z
4760,1299108772296753152,And the same thing can happen with a covid vac...,2020-08-27T22:17:22.000Z
4761,1299108766722347009,@KassandraKitson This isn’t new in most states...,2020-08-27T22:17:20.000Z
4762,1299108749324546051,"As AstraZeneca Signs Deal With EU, Critics War...",2020-08-27T22:17:16.000Z


In [16]:
create_df(2020, 9, 1, 30, 10, 'sept20.csv')

2020-09-02T00:00:00.000Z
Endpoint Response Code: 200
2020-09-04T00:00:00.000Z
Endpoint Response Code: 200
2020-09-07T00:00:00.000Z
Endpoint Response Code: 200
2020-09-10T00:00:00.000Z
Endpoint Response Code: 200
2020-09-13T00:00:00.000Z
Endpoint Response Code: 200
2020-09-16T00:00:00.000Z
Endpoint Response Code: 200
2020-09-19T00:00:00.000Z
Endpoint Response Code: 200
2020-09-22T00:00:00.000Z
Endpoint Response Code: 200
2020-09-25T00:00:00.000Z
Endpoint Response Code: 200
2020-09-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1300946518820696064  @Garold139 @gamestopcorp "Gme pursuing covid v...   
1     1300946505818349569  US refuses to join international effort to dev...   
2     1300946505533214720  The US will not join the WHO-linked global eff...   
3     1300946498243485696  @_computerfreak @ih8evrybdy I’m sure a “I’ve h...   
4     1300946492266512385        Are we the baddies? https://t.co/131m

,id,text,created_at,withheld
0,1300946518820696064,"@Garold139 @gamestopcorp ""Gme pursuing covid v...",2020-09-01T23:59:55.000Z,NaN
1,1300946505818349569,US refuses to join international effort to dev...,2020-09-01T23:59:52.000Z,NaN
2,1300946505533214720,The US will not join the WHO-linked global eff...,2020-09-01T23:59:51.000Z,NaN
3,1300946498243485696,@_computerfreak @ih8evrybdy I’m sure a “I’ve h...,2020-09-01T23:59:50.000Z,NaN
4,1300946492266512385,Are we the baddies? https://t.co/131m8yxD9M,2020-09-01T23:59:48.000Z,NaN
...,...,...,...,...
4787,1310333828104359941,Fourth large-scale COVID-19 vaccine trial begi...,2020-09-27T21:41:44.000Z,NaN
4788,1310333817098469378,dünyada insana gerek var mı?\n\nCoronavirus: H...,2020-09-27T21:41:41.000Z,NaN
4789,1310333737662640129,"Interesting article ""Safety and efficacy of th...",2020-09-27T21:41:22.000Z,NaN
4790,1310333721388691460,Coronavirus: Half a million sharks 'could be k...,2020-09-27T21:41:18.000Z,NaN


In [17]:
create_df(2020, 10, 1, 31, 10, 'oct20.csv')

2020-10-02T00:00:00.000Z
Endpoint Response Code: 200
2020-10-04T00:00:00.000Z
Endpoint Response Code: 200
2020-10-07T00:00:00.000Z
Endpoint Response Code: 200
2020-10-10T00:00:00.000Z
Endpoint Response Code: 200
2020-10-13T00:00:00.000Z
Endpoint Response Code: 200
2020-10-16T00:00:00.000Z
Endpoint Response Code: 200
2020-10-19T00:00:00.000Z
Endpoint Response Code: 200
2020-10-22T00:00:00.000Z
Endpoint Response Code: 200
2020-10-25T00:00:00.000Z
Endpoint Response Code: 200
2020-10-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1311818146848833536  The State Corporation Commission's response to...   
1     1311818119501971456  Moderna's messenger-RNA vaccine to prevent COV...   
2     1311818118256230400                            https://t.co/Hz0hAfqYAy   
3     1311817963469758464  Pfizer CEO says U.S. presidential debate was '...   
4     1311817897468145665  COVID-19 vaccine won't simply end the pande

,id,text,created_at,withheld
0,1311818146848833536,The State Corporation Commission's response to...,2020-10-01T23:59:53.000Z,NaN
1,1311818119501971456,Moderna's messenger-RNA vaccine to prevent COV...,2020-10-01T23:59:46.000Z,NaN
2,1311818118256230400,https://t.co/Hz0hAfqYAy,2020-10-01T23:59:46.000Z,NaN
3,1311817963469758464,Pfizer CEO says U.S. presidential debate was '...,2020-10-01T23:59:09.000Z,NaN
4,1311817897468145665,COVID-19 vaccine won't simply end the pandemic...,2020-10-01T23:58:53.000Z,NaN
...,...,...,...,...
4764,1321225209660215296,CoVPN combines four NIAID-funded clinical tria...,2020-10-27T23:00:11.000Z,NaN
4765,1321225198666805248,We still have the leadership in my state!\nThr...,2020-10-27T23:00:09.000Z,NaN
4766,1321225175153680385,WATCH: COVID-19 vaccine trials resume in #Orla...,2020-10-27T23:00:03.000Z,NaN
4767,1321225173631225856,Pfizer dashes hope for COVID-19 vaccine data b...,2020-10-27T23:00:03.000Z,NaN


In [18]:
create_df(2020, 11, 1, 30, 10, 'nov20.csv')

2020-11-02T00:00:00.000Z
Endpoint Response Code: 200
2020-11-04T00:00:00.000Z
Endpoint Response Code: 200
2020-11-07T00:00:00.000Z
Endpoint Response Code: 200
2020-11-10T00:00:00.000Z
Endpoint Response Code: 200
2020-11-13T00:00:00.000Z
Endpoint Response Code: 200
2020-11-16T00:00:00.000Z
Endpoint Response Code: 200
2020-11-19T00:00:00.000Z
Endpoint Response Code: 200
2020-11-22T00:00:00.000Z
Endpoint Response Code: 200
2020-11-25T00:00:00.000Z
Endpoint Response Code: 200
2020-11-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1323052164063907843  @markknoller https://t.co/qZhrzyuPma\nThe Trum...   
1     1323052156623224832  @MonkeymanKeitho @Care2much18 Antibodies lasti...   
2     1323052065157910528  Sign and send the petition: Demand Congress st...   
3     1323052047646756864  @dedsecluke @Simonr501 @jayar1947 @christopher...   
4     1323052035411947521  @sat1 Corona Alert:  broad virtual/online c

,id,text,created_at,withheld
0,1323052164063907843,@markknoller https://t.co/qZhrzyuPma\nThe Trum...,2020-11-01T23:59:51.000Z,NaN
1,1323052156623224832,@MonkeymanKeitho @Care2much18 Antibodies lasti...,2020-11-01T23:59:49.000Z,NaN
2,1323052065157910528,Sign and send the petition: Demand Congress st...,2020-11-01T23:59:28.000Z,NaN
3,1323052047646756864,@dedsecluke @Simonr501 @jayar1947 @christopher...,2020-11-01T23:59:23.000Z,NaN
4,1323052035411947521,@sat1 Corona Alert: broad virtual/online clin...,2020-11-01T23:59:21.000Z,NaN
...,...,...,...,...
4679,1332465289989074945,NEWS FEED Covid-19: Mass testing a 'distractio...,2020-11-27T23:24:15.000Z,NaN
4680,1332465253892960257,Covid-19: Mass testing a 'distraction' from va...,2020-11-27T23:24:07.000Z,NaN
4681,1332465184087158784,"@JoeBiden We’ve fought nearly 9-mos., America!...",2020-11-27T23:23:50.000Z,NaN
4682,1332465179418710016,Listen to What the Covid Vaccine Hype Fails to...,2020-11-27T23:23:49.000Z,NaN


In [19]:
create_df(2020, 12, 1, 31, 10, 'dec20.csv')

2020-12-02T00:00:00.000Z
Endpoint Response Code: 200
2020-12-04T00:00:00.000Z
Endpoint Response Code: 200
2020-12-07T00:00:00.000Z
Endpoint Response Code: 200
2020-12-10T00:00:00.000Z
Endpoint Response Code: 200
2020-12-13T00:00:00.000Z
Endpoint Response Code: 200
2020-12-16T00:00:00.000Z
Endpoint Response Code: 200
2020-12-19T00:00:00.000Z
Endpoint Response Code: 200
2020-12-22T00:00:00.000Z
Endpoint Response Code: 200
2020-12-25T00:00:00.000Z
Endpoint Response Code: 200
2020-12-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1333923833129041920  Health care workers and nursing home residents...   
1     1333923806570708992  @BillieJeanKing What if the powers that be mad...   
2     1333923787004317696  The Daily Beast: Biden's COVID Team Sounds the...   
3     1333923786530308099  Biden’s COVID Task Force Member Sounds the Ala...   
4     1333923780716990464  A company in America is pushing for an urge

,id,text,created_at,withheld
0,1333923833129041920,Health care workers and nursing home residents...,2020-12-01T23:59:59.000Z,NaN
1,1333923806570708992,@BillieJeanKing What if the powers that be mad...,2020-12-01T23:59:53.000Z,NaN
2,1333923787004317696,The Daily Beast: Biden's COVID Team Sounds the...,2020-12-01T23:59:48.000Z,NaN
3,1333923786530308099,Biden’s COVID Task Force Member Sounds the Ala...,2020-12-01T23:59:48.000Z,NaN
4,1333923780716990464,A company in America is pushing for an urgent ...,2020-12-01T23:59:47.000Z,NaN
...,...,...,...,...
4724,1343340579808305158,Get COVID vaccine even if you caught coronavir...,2020-12-27T23:38:46.000Z,NaN
4725,1343340579162243074,@sparkledocawake @vibrancyrising Poll taken in...,2020-12-27T23:38:46.000Z,NaN
4726,1343340572640215041,#Innovate: Cartoonist’s Take | Putin’s COVID-1...,2020-12-27T23:38:45.000Z,NaN
4727,1343340555355496449,"CVS, Walgreens to begin administering COVID-19...",2020-12-27T23:38:41.000Z,NaN


In [20]:
create_df(2021, 1, 1, 31, 10, 'jan21.csv')

2021-01-02T00:00:00.000Z
Endpoint Response Code: 200
2021-01-04T00:00:00.000Z
Endpoint Response Code: 200
2021-01-07T00:00:00.000Z
Endpoint Response Code: 200
2021-01-10T00:00:00.000Z
Endpoint Response Code: 200
2021-01-13T00:00:00.000Z
Endpoint Response Code: 200
2021-01-16T00:00:00.000Z
Endpoint Response Code: 200
2021-01-19T00:00:00.000Z
Endpoint Response Code: 200
2021-01-22T00:00:00.000Z
Endpoint Response Code: 200
2021-01-25T00:00:00.000Z
Endpoint Response Code: 200
2021-01-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1345157845763952646  @cowboythoughtss tell him the covid vaccine tu...   
1     1345157807348318208  @NigilHaroon @CRASCRRheum @MakeRheum @spondyli...   
2     1345157766655188992  @MaxCRoser Please don't praise Israel more tha...   
3     1345157762158915585  @BpsmithUk @sianjames1 We knew about this back...   
4     1345157756878278657  WHO still dey exist? #EndSars #SoroSoke htt

,id,text,created_at,withheld
0,1345157845763952646,@cowboythoughtss tell him the covid vaccine tu...,2021-01-01T23:59:56.000Z,NaN
1,1345157807348318208,@NigilHaroon @CRASCRRheum @MakeRheum @spondyli...,2021-01-01T23:59:47.000Z,NaN
2,1345157766655188992,@MaxCRoser Please don't praise Israel more tha...,2021-01-01T23:59:38.000Z,NaN
3,1345157762158915585,@BpsmithUk @sianjames1 We knew about this back...,2021-01-01T23:59:36.000Z,NaN
4,1345157756878278657,WHO still dey exist? #EndSars #SoroSoke https:...,2021-01-01T23:59:35.000Z,NaN
...,...,...,...,...
4667,1354576073804177410,Wilton’s Success (So Far) with COVID Vaccine C...,2021-01-27T23:44:37.000Z,NaN
4668,1354576068041207814,"Publix gives $100,000 to DeSantis, gets exclus...",2021-01-27T23:44:36.000Z,NaN
4669,1354576055512821762,A good idea for a change! Wonder why this didn...,2021-01-27T23:44:33.000Z,NaN
4670,1354576047510052864,It is unacceptable that MD's communities of co...,2021-01-27T23:44:31.000Z,NaN


In [21]:
create_df(2021, 2, 1, 28, 10, 'feb21.csv')

2021-02-02T00:00:00.000Z
Endpoint Response Code: 200
2021-02-04T00:00:00.000Z
Endpoint Response Code: 200
2021-02-07T00:00:00.000Z
Endpoint Response Code: 200
2021-02-10T00:00:00.000Z
Endpoint Response Code: 200
2021-02-13T00:00:00.000Z
Endpoint Response Code: 200
2021-02-16T00:00:00.000Z
Endpoint Response Code: 200
2021-02-19T00:00:00.000Z
Endpoint Response Code: 200
2021-02-22T00:00:00.000Z
Endpoint Response Code: 200
2021-02-25T00:00:00.000Z
Endpoint Response Code: 200
2021-02-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1356391881437892611  With the rollout of a COVID-19 vaccine seeming...   
1     1356391881131757568  Here is today’s Idaho COVID-19 vaccine data at...   
2     1356391876836884480  Health care worker dies after second dose of C...   
3     1356391851499151371  Boris Johnson: Emergency Services to receive C...   
4     1356391843039219712  @Jen70717630 "COVID-19 mRNA Vaccine BNT162b

,id,text,created_at,withheld
0,1356391881437892611,With the rollout of a COVID-19 vaccine seeming...,2021-02-01T23:59:59.000Z,NaN
1,1356391881131757568,Here is today’s Idaho COVID-19 vaccine data at...,2021-02-01T23:59:59.000Z,NaN
2,1356391876836884480,Health care worker dies after second dose of C...,2021-02-01T23:59:58.000Z,NaN
3,1356391851499151371,Boris Johnson: Emergency Services to receive C...,2021-02-01T23:59:52.000Z,NaN
4,1356391843039219712,"@Jen70717630 ""COVID-19 mRNA Vaccine BNT162b2 i...",2021-02-01T23:59:50.000Z,NaN
...,...,...,...,...
4803,1365811737736470533,FDA grants emergency use authorization for Joh...,2021-02-27T23:51:08.000Z,NaN
4804,1365811734477492232,JUST IN: The FDA authorized Johnson and Johnso...,2021-02-27T23:51:07.000Z,NaN
4805,1365811729188462598,Johnson &amp; Johnson's COVID-19 vaccine gets ...,2021-02-27T23:51:06.000Z,NaN
4806,1365811726197776385,This is beyond disturbing. \n\nhttps://t.co/8Z...,2021-02-27T23:51:05.000Z,NaN


In [22]:
create_df(2021, 3, 1, 31, 10, 'march21.csv')

2021-03-02T00:00:00.000Z
Endpoint Response Code: 200
2021-03-04T00:00:00.000Z
Endpoint Response Code: 200
2021-03-07T00:00:00.000Z
Endpoint Response Code: 200
2021-03-10T00:00:00.000Z
Endpoint Response Code: 200
2021-03-13T00:00:00.000Z
Endpoint Response Code: 200
2021-03-16T00:00:00.000Z
Endpoint Response Code: 200
2021-03-19T00:00:00.000Z
Endpoint Response Code: 200
2021-03-22T00:00:00.000Z
Endpoint Response Code: 200
2021-03-25T00:00:00.000Z
Endpoint Response Code: 200
2021-03-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1366538730593345537  REMINDER: THOMASTON HIGH SCHOOL DISMISSING EAR...   
1     1366538728521404423  Olympic Gold Medalist Says He’d Rather Not Com...   
2     1366538727179255810  Hey ⁦@GovLarryHogan⁩, #Maryland is near the bo...   
3     1366538727120465921  Olympic Gold Medalist Says He’d Rather Not Com...   
4     1366538708632150016  Covid vaccine update: My mom confirms that 

,id,text,created_at
0,1366538730593345537,REMINDER: THOMASTON HIGH SCHOOL DISMISSING EAR...,2021-03-01T23:59:57.000Z
1,1366538728521404423,Olympic Gold Medalist Says He’d Rather Not Com...,2021-03-01T23:59:56.000Z
2,1366538727179255810,"Hey ⁦@GovLarryHogan⁩, #Maryland is near the bo...",2021-03-01T23:59:56.000Z
3,1366538727120465921,Olympic Gold Medalist Says He’d Rather Not Com...,2021-03-01T23:59:56.000Z
4,1366538708632150016,Covid vaccine update: My mom confirms that sin...,2021-03-01T23:59:51.000Z
...,...,...,...
4816,1375953132467806208,@kinsellawarren Where’d you find that? I see 4...,2021-03-27T23:29:25.000Z
4817,1375953127631884290,All Madison County residents 16 or older now e...,2021-03-27T23:29:24.000Z
4818,1375953122426675201,@Conan644 @DrNinaRadcliff I've been saying the...,2021-03-27T23:29:23.000Z
4819,1375953120795230213,@ElizabethBangs @foodologistgirl @David07982 @...,2021-03-27T23:29:22.000Z


In [23]:
create_df(2021, 4, 1, 30, 10, 'april21.csv')

2021-04-02T00:00:00.000Z
Endpoint Response Code: 200
2021-04-04T00:00:00.000Z
Endpoint Response Code: 200
2021-04-07T00:00:00.000Z
Endpoint Response Code: 200
2021-04-10T00:00:00.000Z
Endpoint Response Code: 200
2021-04-13T00:00:00.000Z
Endpoint Response Code: 200
2021-04-16T00:00:00.000Z
Endpoint Response Code: 200
2021-04-19T00:00:00.000Z
Endpoint Response Code: 200
2021-04-22T00:00:00.000Z
Endpoint Response Code: 200
2021-04-25T00:00:00.000Z
Endpoint Response Code: 200
2021-04-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1377772757841289216  @EssexPR @LBC Not everyone, myself included, w...   
1     1377772757275004931  As of Mar 29, more than 3,325,998 doses of vac...   
2     1377772750060814336  The one-shot vaccine dose is expected to be a ...   
3     1377772739667238915  Covid Vaccine Drive Thru Protocol/Soka Univers...   
4     1377772736957837312  I JUST GOT MY COVID VACCINE APPOINTMENT YES

,id,text,created_at
0,1377772757841289216,"@EssexPR @LBC Not everyone, myself included, w...",2021-04-01T23:59:57.000Z
1,1377772757275004931,"As of Mar 29, more than 3,325,998 doses of vac...",2021-04-01T23:59:57.000Z
2,1377772750060814336,The one-shot vaccine dose is expected to be a ...,2021-04-01T23:59:56.000Z
3,1377772739667238915,Covid Vaccine Drive Thru Protocol/Soka Univers...,2021-04-01T23:59:53.000Z
4,1377772736957837312,I JUST GOT MY COVID VACCINE APPOINTMENT YESSSS...,2021-04-01T23:59:52.000Z
...,...,...,...
4796,1387190925231902723,Please fill out this survey being done by @uni...,2021-04-27T23:44:23.000Z
4797,1387190883427225601,Did you get the COVID-19 vaccine?,2021-04-27T23:44:14.000Z
4798,1387190880159817728,@crustysails @JordanArchule18 @RealCandaceO @T...,2021-04-27T23:44:13.000Z
4799,1387190880143040513,"This past Friday on The Joe Rogan Experience, ...",2021-04-27T23:44:13.000Z


In [24]:
create_df(2021, 5, 1, 31, 10, 'may21.csv')

2021-05-02T00:00:00.000Z
Endpoint Response Code: 200
2021-05-04T00:00:00.000Z
Endpoint Response Code: 200
2021-05-07T00:00:00.000Z
Endpoint Response Code: 200
2021-05-10T00:00:00.000Z
Endpoint Response Code: 200
2021-05-13T00:00:00.000Z
Endpoint Response Code: 200
2021-05-16T00:00:00.000Z
Endpoint Response Code: 200
2021-05-19T00:00:00.000Z
Endpoint Response Code: 200
2021-05-22T00:00:00.000Z
Endpoint Response Code: 200
2021-05-25T00:00:00.000Z
Endpoint Response Code: 200
2021-05-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1388644400088092673  N.J. could get 70K doses of Johnson &amp; John...   
1     1388644341963366404  @santoshkrshaw9 @akhand_9051 @huqian82 @zlj517...   
2     1388644321407217670  From designing production facilities to conque...   
3     1388644321122000900  @hddsouza @BogochIsaac "Review of available da...   
4     1388644311949053967  @MarshaBlackburn If your “leader” had not l

,id,text,created_at
0,1388644400088092673,N.J. could get 70K doses of Johnson &amp; John...,2021-05-01T23:59:59.000Z
1,1388644341963366404,@santoshkrshaw9 @akhand_9051 @huqian82 @zlj517...,2021-05-01T23:59:45.000Z
2,1388644321407217670,From designing production facilities to conque...,2021-05-01T23:59:40.000Z
3,1388644321122000900,"@hddsouza @BogochIsaac ""Review of available da...",2021-05-01T23:59:40.000Z
4,1388644311949053967,@MarshaBlackburn If your “leader” had not lied...,2021-05-01T23:59:38.000Z
...,...,...,...
4811,1398060963727917056,@bea_johanssen @LynnBrittney2 Link to the full...,2021-05-27T23:38:03.000Z
4812,1398060963148951555,@CMOH_Alberta Will you please consider opening...,2021-05-27T23:38:02.000Z
4813,1398060958879154179,"@healthgovau ""To 23 May 2021, 3.6 million dose...",2021-05-27T23:38:01.000Z
4814,1398060943020478465,A very informative article which visually show...,2021-05-27T23:37:58.000Z


In [25]:
create_df(2021, 6, 1, 30, 10, 'june21.csv')

2021-06-02T00:00:00.000Z
Endpoint Response Code: 200
2021-06-04T00:00:00.000Z
Endpoint Response Code: 200
2021-06-07T00:00:00.000Z
Endpoint Response Code: 200
2021-06-10T00:00:00.000Z
Endpoint Response Code: 200
2021-06-13T00:00:00.000Z
Endpoint Response Code: 200
2021-06-16T00:00:00.000Z
Endpoint Response Code: 200
2021-06-19T00:00:00.000Z
Endpoint Response Code: 200
2021-06-22T00:00:00.000Z
Endpoint Response Code: 200
2021-06-25T00:00:00.000Z
Endpoint Response Code: 200
2021-06-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1399878422961156101  @PublicDefende21 @LiVe2DaNceATL You're right. ...   
1     1399878420222287874  “Data released Friday by the CDC Vaccine Adver...   
2     1399878380594470914  Mobile vaccination clinic bringing shots to Ha...   
3     1399878376534560768  One World #ChinaVirus ”world treaty” #G7 “vacc...   
4     1399878351532314624  Here is one of the little-discussed aspects

,id,text,created_at
0,1399878422961156101,@PublicDefende21 @LiVe2DaNceATL You're right. ...,2021-06-01T23:59:59.000Z
1,1399878420222287874,“Data released Friday by the CDC Vaccine Adver...,2021-06-01T23:59:58.000Z
2,1399878380594470914,Mobile vaccination clinic bringing shots to Ha...,2021-06-01T23:59:49.000Z
3,1399878376534560768,One World #ChinaVirus ”world treaty” #G7 “vacc...,2021-06-01T23:59:48.000Z
4,1399878351532314624,Here is one of the little-discussed aspects of...,2021-06-01T23:59:42.000Z
...,...,...,...
4900,1409290700517167110,COVID-19 Vaccination appointments available at...,2021-06-27T23:21:01.000Z
4901,1409290700487725063,COVID-19 Vaccination appointments available at...,2021-06-27T23:21:01.000Z
4902,1409290700437471234,COVID-19 Vaccination appointments available at...,2021-06-27T23:21:00.000Z
4903,1409290700429139970,COVID-19 Vaccination appointments available at...,2021-06-27T23:21:00.000Z


In [26]:
create_df(2021, 7, 1, 31, 10, 'july21.csv')

2021-07-02T00:00:00.000Z
Endpoint Response Code: 200
2021-07-04T00:00:00.000Z
Endpoint Response Code: 200
2021-07-07T00:00:00.000Z
Endpoint Response Code: 200
2021-07-10T00:00:00.000Z
Endpoint Response Code: 200
2021-07-13T00:00:00.000Z
Endpoint Response Code: 200
2021-07-16T00:00:00.000Z
Endpoint Response Code: 200
2021-07-19T00:00:00.000Z
Endpoint Response Code: 200
2021-07-22T00:00:00.000Z
Endpoint Response Code: 200
2021-07-25T00:00:00.000Z
Endpoint Response Code: 200
2021-07-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1410750052868321281  @DbMistafunk @Artemitzi Vaccine hesitancy seem...   
1     1410749985579180046  Positive New Data for Johnson &amp; Johnson Si...   
2     1410749982144045060  $JNJ Positive New Data for Johnson &amp; Johns...   
3     1410749965031251974  Vaccine mania led to all-out suppression of CO...   
4     1410749952376987649  CureVac Covid-19 vaccine records only 48% e

,id,text,created_at
0,1410750052868321281,@DbMistafunk @Artemitzi Vaccine hesitancy seem...,2021-07-01T23:59:57.000Z
1,1410749985579180046,Positive New Data for Johnson &amp; Johnson Si...,2021-07-01T23:59:41.000Z
2,1410749982144045060,$JNJ Positive New Data for Johnson &amp; Johns...,2021-07-01T23:59:40.000Z
3,1410749965031251974,Vaccine mania led to all-out suppression of CO...,2021-07-01T23:59:36.000Z
4,1410749952376987649,CureVac Covid-19 vaccine records only 48% effi...,2021-07-01T23:59:33.000Z
...,...,...,...
4918,1420168408193114112,Everyone that got covid after they got the vac...,2021-07-27T23:45:08.000Z
4919,1420168405626376193,Does anyone have a good explainer/video of why...,2021-07-27T23:45:08.000Z
4920,1420168397405491203,The C.D.C.'s revised mask guidance may lead ma...,2021-07-27T23:45:06.000Z
4921,1420168388698021890,Moderna to resume COVID-19 vaccine shipments t...,2021-07-27T23:45:03.000Z


In [27]:
create_df(2021, 8, 1, 31, 10, 'august21.csv')

2021-08-02T00:00:00.000Z
Endpoint Response Code: 200
2021-08-04T00:00:00.000Z
Endpoint Response Code: 200
2021-08-07T00:00:00.000Z
Endpoint Response Code: 200
2021-08-10T00:00:00.000Z
Endpoint Response Code: 200
2021-08-13T00:00:00.000Z
Endpoint Response Code: 200
2021-08-16T00:00:00.000Z
Endpoint Response Code: 200
2021-08-19T00:00:00.000Z
Endpoint Response Code: 200
2021-08-22T00:00:00.000Z
Endpoint Response Code: 200
2021-08-25T00:00:00.000Z
Endpoint Response Code: 200
2021-08-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1421984081491939333  @prothoughts2 ADE Will murder people injected ...   
1     1421984045706235911  @jrf195959 @hillsbarbers @JeromeAdamsMD No one...   
2     1421984043520909312  Data on the Covid-19 vaccine shows that you ha...   
3     1421984028706709505  It’s crazy how mfs went from “Covid isn’t real...   
4     1421983998369095681  @KaseyGirlBye @NoyahKing And before you ask

,id,text,created_at
0,1421984081491939333,@prothoughts2 ADE Will murder people injected ...,2021-08-01T23:59:58.000Z
1,1421984045706235911,@jrf195959 @hillsbarbers @JeromeAdamsMD No one...,2021-08-01T23:59:50.000Z
2,1421984043520909312,Data on the Covid-19 vaccine shows that you ha...,2021-08-01T23:59:49.000Z
3,1421984028706709505,It’s crazy how mfs went from “Covid isn’t real...,2021-08-01T23:59:46.000Z
4,1421983998369095681,@KaseyGirlBye @NoyahKing And before you ask fo...,2021-08-01T23:59:39.000Z
...,...,...,...
4963,1431402383037325315,Prescription: Looking for a job? Montana bans ...,2021-08-27T23:44:56.000Z
4964,1431402378184597510,@JacquiDeevoy1 Not a troll here. I got the vac...,2021-08-27T23:44:55.000Z
4965,1431402365500854274,Authorised workers have just over a week to ge...,2021-08-27T23:44:52.000Z
4966,1431402363101843457,@MarkBednar @GOPLeader Joe &amp; Kamala's\nHHS...,2021-08-27T23:44:52.000Z


In [28]:
create_df(2021, 9, 1, 30, 10, 'sept21.csv')

2021-09-02T00:00:00.000Z
Endpoint Response Code: 200
2021-09-04T00:00:00.000Z
Endpoint Response Code: 200
2021-09-07T00:00:00.000Z
Endpoint Response Code: 200
2021-09-10T00:00:00.000Z
Endpoint Response Code: 200
2021-09-13T00:00:00.000Z
Endpoint Response Code: 200
2021-09-16T00:00:00.000Z
Endpoint Response Code: 200
2021-09-19T00:00:00.000Z
Endpoint Response Code: 200
2021-09-22T00:00:00.000Z
Endpoint Response Code: 200
2021-09-25T00:00:00.000Z
Endpoint Response Code: 200
2021-09-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1433218100791500804  COVID-19 vaccine required for anyone working a...   
1     1433218098849406977  “Duke enacted a vaccine mandate for staff and ...   
2     1433218086086184965  More than 15 million coronavirus vaccine doses...   
3     1433218077655584768  Beltway Insider: Biden/Kabul/ISIS Attack, Hoch...   
4     1433218046848638976  Ask for COVID vaccine proof, face a $5,000 

,id,text,created_at
0,1433218100791500804,COVID-19 vaccine required for anyone working a...,2021-09-01T23:59:57.000Z
1,1433218098849406977,“Duke enacted a vaccine mandate for staff and ...,2021-09-01T23:59:57.000Z
2,1433218086086184965,More than 15 million coronavirus vaccine doses...,2021-09-01T23:59:54.000Z
3,1433218077655584768,"Beltway Insider: Biden/Kabul/ISIS Attack, Hoch...",2021-09-01T23:59:52.000Z
4,1433218046848638976,"Ask for COVID vaccine proof, face a $5,000 fin...",2021-09-01T23:59:44.000Z
...,...,...,...
4963,1442636144571666432,"Beltway Insider: Biden/UN, France, Canada, COV...",2021-09-27T23:43:54.000Z
4964,1442636139328942083,Looking to book your vaccine or any other COVI...,2021-09-27T23:43:53.000Z
4965,1442636106290372609,Dozens Of Massachusetts State Police Troopers ...,2021-09-27T23:43:45.000Z
4966,1442636105933807617,@mminymo Just bc ur unvaccinated doesn't mean ...,2021-09-27T23:43:45.000Z


In [29]:
create_df(2021, 10, 1, 31, 10, 'oct21.csv')

2021-10-02T00:00:00.000Z
Endpoint Response Code: 200
2021-10-04T00:00:00.000Z
Endpoint Response Code: 200
2021-10-07T00:00:00.000Z
Endpoint Response Code: 200
2021-10-10T00:00:00.000Z
Endpoint Response Code: 200
2021-10-13T00:00:00.000Z
Endpoint Response Code: 200
2021-10-16T00:00:00.000Z
Endpoint Response Code: 200
2021-10-19T00:00:00.000Z
Endpoint Response Code: 200
2021-10-22T00:00:00.000Z
Endpoint Response Code: 200
2021-10-25T00:00:00.000Z
Endpoint Response Code: 200
2021-10-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1444089745953497090  @HughBoone9 @Teddys_Mum @GauteNilsen not if th...   
1     1444089743130648585  Biden against giving honorable discharge to tr...   
2     1444089728320647171  @AndrewW09454228 @JFreshHockey Thousands of pr...   
3     1444089726890356739  Covid vaccine side effects are not being recor...   
4     1444089721341235208  @aftfarm @CTVNews The Covid shot, unlike a 

,id,text,created_at
0,1444089745953497090,@HughBoone9 @Teddys_Mum @GauteNilsen not if th...,2021-10-01T23:59:59.000Z
1,1444089743130648585,Biden against giving honorable discharge to tr...,2021-10-01T23:59:59.000Z
2,1444089728320647171,@AndrewW09454228 @JFreshHockey Thousands of pr...,2021-10-01T23:59:55.000Z
3,1444089726890356739,Covid vaccine side effects are not being recor...,2021-10-01T23:59:55.000Z
4,1444089721341235208,"@aftfarm @CTVNews The Covid shot, unlike a rea...",2021-10-01T23:59:54.000Z
...,...,...,...
4946,1453506603433426955,"As a CA resident, I don’t get why people even ...",2021-10-27T23:39:13.000Z
4947,1453506592897331203,"as far as i can tell, people are big mad about...",2021-10-27T23:39:11.000Z
4948,1453506581631488010,Belgian minister of health Frank Vandenbroucke...,2021-10-27T23:39:08.000Z
4949,1453506556709023749,While the latest recommendations on boosters f...,2021-10-27T23:39:02.000Z


In [30]:
create_df(2021, 11, 1, 30, 10, 'nov21.csv')

2021-11-02T00:00:00.000Z
Endpoint Response Code: 200
2021-11-04T00:00:00.000Z
Endpoint Response Code: 200
2021-11-07T00:00:00.000Z
Endpoint Response Code: 200
2021-11-10T00:00:00.000Z
Endpoint Response Code: 200
2021-11-13T00:00:00.000Z
Endpoint Response Code: 200
2021-11-16T00:00:00.000Z
Endpoint Response Code: 200
2021-11-19T00:00:00.000Z
Endpoint Response Code: 200
2021-11-22T00:00:00.000Z
Endpoint Response Code: 200
2021-11-25T00:00:00.000Z
Endpoint Response Code: 200
2021-11-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1455323760182059012  @Bebert701 @andylumm @BrowningNeelyn @F250Rous...   
1     1455323733191712771  @briantylercohen Love how they all ask for pra...   
2     1455323729261514753  EXCLUSIVE – 100% of Covid-19 Vaccine Deaths we...   
3     1455323696021876737  @ChrisSturr @peronihoe @onearmedsteve69 @Gallo...   
4     1455323676220395520  @vuelvealahora @EconomistaFlait @b0mb3r0 ht

,id,text,created_at
0,1455323760182059012,@Bebert701 @andylumm @BrowningNeelyn @F250Rous...,2021-11-01T23:59:57.000Z
1,1455323733191712771,@briantylercohen Love how they all ask for pra...,2021-11-01T23:59:51.000Z
2,1455323729261514753,EXCLUSIVE – 100% of Covid-19 Vaccine Deaths we...,2021-11-01T23:59:50.000Z
3,1455323696021876737,@ChrisSturr @peronihoe @onearmedsteve69 @Gallo...,2021-11-01T23:59:42.000Z
4,1455323676220395520,@vuelvealahora @EconomistaFlait @b0mb3r0 https...,2021-11-01T23:59:37.000Z
...,...,...,...
4928,1464738312329961472,Punished!! What a load.. Get your vaccine rate...,2021-11-27T23:30:01.000Z
4929,1464738303865982977,via ⁦@nytimes⁩ \nDo not patent the Vaccine for...,2021-11-27T23:29:59.000Z
4930,1464738297268240384,@Stonekettle I actually don't understand why y...,2021-11-27T23:29:58.000Z
4931,1464738285117390852,COVID-19 mRNA Vaccination in Lactation: Assess...,2021-11-27T23:29:55.000Z


In [31]:
create_df(2021, 12, 1, 31, 10, 'dec21.csv')

2021-12-02T00:00:00.000Z
Endpoint Response Code: 200
2021-12-04T00:00:00.000Z
Endpoint Response Code: 200
2021-12-07T00:00:00.000Z
Endpoint Response Code: 200
2021-12-10T00:00:00.000Z
Endpoint Response Code: 200
2021-12-13T00:00:00.000Z
Endpoint Response Code: 200
2021-12-16T00:00:00.000Z
Endpoint Response Code: 200
2021-12-19T00:00:00.000Z
Endpoint Response Code: 200
2021-12-22T00:00:00.000Z
Endpoint Response Code: 200
2021-12-25T00:00:00.000Z
Endpoint Response Code: 200
2021-12-28T00:00:00.000Z
Endpoint Response Code: 200
                       id                                               text  \
0     1466195393998467072  @DictionaryGo @freep Not celebrating but findi...   
1     1466195390735151106  @kenklippenstein @nycsouthpaw "Despite the fac...   
2     1466195346850463744  They should run on this. Republicans: Let Peop...   
3     1466195346648875009  Another one down the tubes👍👍👍👍\nhttps://t.co/C...   
4     1466195341125029899  @druidian @mike_of_oz @Breedlove22 So, why 

,id,text,created_at
0,1466195393998467072,@DictionaryGo @freep Not celebrating but findi...,2021-12-01T23:59:57.000Z
1,1466195390735151106,"@kenklippenstein @nycsouthpaw ""Despite the fac...",2021-12-01T23:59:56.000Z
2,1466195346850463744,They should run on this. Republicans: Let Peop...,2021-12-01T23:59:45.000Z
3,1466195346648875009,Another one down the tubes👍👍👍👍\nhttps://t.co/C...,2021-12-01T23:59:45.000Z
4,1466195341125029899,"@druidian @mike_of_oz @Breedlove22 So, why Tru...",2021-12-01T23:59:44.000Z
...,...,...,...
4926,1475612500716670985,@doctor_regtools @jlbradstreet @chosenone_inni...,2021-12-27T23:40:10.000Z
4927,1475612494563446785,@MeidasTouch @mtgreenee Funny how they zero in...,2021-12-27T23:40:08.000Z
4928,1475612483066810368,I recently wrote about Petrovsky too following...,2021-12-27T23:40:06.000Z
4929,1475612452805091333,@MakeItPersonal_ @DylanWa77597613 @dsleon45 Ac...,2021-12-27T23:39:58.000Z
